Import neccessary packages

In [1]:
import numpy as np
from os import listdir
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth 
import google.colab
from oauth2client.client import GoogleCredentials
from keras.applications import ResNet50V2
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()

In [0]:
drive = GoogleDrive(gauth)


In [0]:
EPOCHS = 15
INIT_LR = 1e-3
batch_size = 32
default_image_size = tuple((224, 224))
image_size = 0
directory_root = '/content/PlantVillage-Dataset/raw/color'
width=224
height=224
depth=3

In [5]:
!git clone https://github.com/spMohanty/PlantVillage-Dataset.git

Cloning into 'PlantVillage-Dataset'...
remote: Enumerating objects: 163229, done.
remote: Total 163229 (delta 0), reused 0 (delta 0), pack-reused 163229
Receiving objects: 100% (163229/163229), 2.00 GiB | 35.46 MiB/s, done.
Resolving deltas: 100% (99/99), done.
Checking out files: 100% (182401/182401), done.


In [6]:
import os
os.getcwd()
%ls

adc.json  PlantVillage-Dataset/  sample_data/


Fetch images from directory

In [7]:
listdir(f"{directory_root}")

['Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Cherry_(including_sour)___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Apple___Apple_scab',
 'Strawberry___Leaf_scorch',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Apple___Cedar_apple_rust',
 'Tomato___healthy',
 'Corn_(maize)___healthy',
 'Pepper,_bell___healthy',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Soybean___healthy',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Raspberry___healthy',
 'Grape___Esca_(Black_Measles)',
 'Peach___Bacterial_spot',
 'Tomato___Leaf_Mold',
 'Tomato___Bacterial_spot',
 'Squash___Powdery_mildew',
 'Tomato___Late_blight',
 'Apple___healthy',
 'Tomato___Tomato_mosaic_virus',
 'Blueberry___healthy',
 'Corn_(maize)___Common_rust_',
 'Tomato___Target_Spot',
 'Tomato___Septoria_leaf_spot',
 'Potato___healthy',
 'Apple___Black_rot',
 'Strawberry___healthy',
 'Peach___healthy',
 'Grape___Black_rot',
 'Tomato___Early_blight',
 'Grape___healthy',
 'Cherry_(including_sour)___Powdery_milde

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    directory_root,
    target_size=(height, width),
    batch_size=batch_size,
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    directory_root, # same directory as training data
    target_size=(height, width),
    batch_size=batch_size,
    subset='validation') # set as validation data


Found 43456 images belonging to 38 classes.
Found 10849 images belonging to 38 classes.


Get Size of Processed Image

In [9]:
model = ResNet50V2(classes = 38, weights=None)

Model Summary

In [10]:
model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

In [11]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the network
callbacks = [EarlyStopping(monitor='val_loss', patience=2),
             ModelCheckpoint(filepath='resnetv2_50_plant.h5', monitor='val_loss', save_best_only=True)]
print("[INFO] training network...")



[INFO] training network...


In [0]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = EPOCHS)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/15
1358/1358 [==============================] - 1187s 874ms/step - loss: 1.1195 - acc: 0.6684 - val_loss: 2.1117 - val_acc: 0.5383
Epoch 2/15
1358/1358 [==============================] - 1171s 863ms/step - loss: 0.4263 - acc: 0.8620 - val_loss: 1.5469 - val_acc: 0.6295
Epoch 3/15
1358/1358 [==============================] - 1166s 859ms/step - loss: 0.2787 - acc: 0.9084 - val_loss: 1.1567 - val_acc: 0.7120
Epoch 4/15
1358/1358 [==============================] - 1172s 863ms/step - loss: 0.2069 - acc: 0.9324 - val_loss: 2.0309 - val_acc: 0.6090
Epoch 5/15
1358/1358 [==============================] - 1170s 861ms/step - loss: 0.1651 - acc: 0.9439 - val_loss: 0.8658 - val_acc: 0.7797
Epoch 6/15
1358/1358 [==============================] - 1162s 856ms/step - loss: 0.1281 - acc: 0.9573 - val_loss: 0.2232 - val_acc: 0.9254
Epoch 7/15
1358/1358 [==============================] - 1164s 857ms/st

In [0]:
!mv resnetv2_50_plant.h5 gdrive/My\ Drive/

Plot the train and val curve

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

Model Accuracy

In [0]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")